# 🌸 วิเคราะห์ข้อมูลชุด Iris และสร้างโมเดล KNN อย่างละเอียด
## 📘 จุดประสงค์: เรียนรู้วิธีการวิเคราะห์ข้อมูลและสร้างโมเดล Machine Learning เบื้องต้นด้วย K-Nearest Neighbors (KNN)

## 1. 🔍 ข้อมูลชุด Iris คืออะไร?
ชุดข้อมูล Iris ถูกใช้ครั้งแรกโดยนักสถิติชื่อว่า **Ronald A. Fisher** ในปี 1936 เพื่อแสดงตัวอย่างการจำแนกประเภท (Classification)

ในชุดข้อมูลประกอบด้วยข้อมูลของดอกไม้ 3 สายพันธุ์ ได้แก่:
- Iris setosa
- Iris versicolor
- Iris virginica

แต่ละตัวอย่างจะมีคุณลักษณะ (Feature) จำนวน 4 ตัว คือ:
- ความยาวกลีบดอกด้านนอก (sepal length)
- ความกว้างกลีบดอกด้านนอก (sepal width)
- ความยาวกลีบดอกด้านใน (petal length)
- ความกว้างกลีบดอกด้านใน (petal width)

และมี **label** หรือ target class เป็นชื่อสายพันธุ์ของดอกไม้

## 2. 📂 โหลดข้อมูลจากไฟล์
เราจะโหลดข้อมูลจากไฟล์ `.data` ที่ไม่มีชื่อคอลัมน์ (header) และเราต้องตั้งชื่อคอลัมน์เองเพื่อความสะดวกในการใช้งาน

In [ ]:
pip install --upgrade -r library_name.txt

In [ ]:
import pandas as pd

# ตั้งชื่อคอลัมน์เอง
column_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df = pd.read_csv(r"C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_OTHER\0004\iris\bezdekIris.data", header=None, names=column_names)
df.head()

## 🌿 รายละเอียดของ Features ทั้ง 4 ตัวในชุดข้อมูล Iris
ข้อมูลของดอกไม้แต่ละตัวอย่างในชุดข้อมูลนี้มี 4 ตัวแปร (features) ดังนี้:

1. **sepal_length** (ความยาวกลีบเลี้ยง) - วัดเป็นหน่วยเซนติเมตร
2. **sepal_width** (ความกว้างกลีบเลี้ยง) - วัดเป็นหน่วยเซนติเมตร
3. **petal_length** (ความยาวกลีบดอก) - วัดเป็นหน่วยเซนติเมตร
4. **petal_width** (ความกว้างกลีบดอก	) - วัดเป็นหน่วยเซนติเมตร

> โดยปกติแล้ว `petal_length` และ `petal_width` มักเป็นฟีเจอร์ที่แยกความแตกต่างของสายพันธุ์ได้ดีที่สุด
> ขณะที่ `sepal_width` บางครั้งอาจมีการกระจายที่ซ้อนกันระหว่างสายพันธุ์

## 3. 🧼 ตรวจสอบข้อมูลเบื้องต้น
เราจะดูโครงสร้างข้อมูล เช่น จำนวนแถว-คอลัมน์, ประเภทข้อมูล, และสถิติเบื้องต้น
สิ่งนี้ช่วยให้เราเข้าใจว่าข้อมูลพร้อมใช้หรือไม่ และต้องเตรียมอะไรเพิ่มเติมหรือไม่

In [ ]:
df.info()

In [ ]:
df.describe()

## 4. ✂️ เตรียมข้อมูลสำหรับ Machine Learning
Machine Learning ต้องการข้อมูลในรูปแบบตัวเลข
- เราจะแยก feature (X) และ target class (y)
- แล้วแปลง label จากข้อความ (ชื่อสายพันธุ์) เป็นตัวเลข (0, 1, 2)
- จากนั้นแบ่งข้อมูลออกเป็นชุด train (ฝึกสอน) และ test (ทดสอบ)

### 🔄 การแบ่งข้อมูลออกเป็นชุดฝึก (Train set) และชุดทดสอบ (Test set)
หลังจากที่เราเตรียมข้อมูล X และ y แล้ว ขั้นตอนต่อไปคือการแบ่งข้อมูลออกเป็น 2 ส่วน:

- **Train set**: ใช้สำหรับฝึกโมเดล (เรียนรู้จากข้อมูล)
- **Test set**: ใช้สำหรับทดสอบความสามารถของโมเดลที่เรียนรู้แล้ว

โดยทั่วไปนิยมแบ่งข้อมูลเป็น **80% สำหรับ Train และ 20% สำหรับ Test**

📌 ทั้งสองชุดจะยังคงมี **ฟีเจอร์ 4 ตัวเท่ากัน** ได้แก่:
`sepal_length`, `sepal_width`, `petal_length`, `petal_width`

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# แยกข้อมูล features และ labels
X = df.iloc[:, :-1]  # คอลัมน์ 0-3 เป็น features
y = df['class']     # คอลัมน์สุดท้ายเป็น label

# แปลง label เป็นตัวเลข
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# แบ่งข้อมูลเป็น train (80%) และ test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# แสดงจำนวนข้อมูลในแต่ละชุด
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

In [ ]:
# แสดงตัวอย่างข้อมูลใน train/test set เพื่อยืนยันว่าแต่ละชุดมี 4 ฟีเจอร์
print("🔹 Train set:")
display(X_train.head())
print("\n🔹 Test set:")
display(X_test.head())

## 5. 🧠 สร้างโมเดล K-Nearest Neighbors (KNN)
**KNN** เป็นโมเดลแบบง่ายที่ใช้การวัด "ระยะทาง" ระหว่างจุดข้อมูลเพื่อทำนายคลาสของข้อมูลใหม่
- เราเลือก `k = 3` หมายถึงให้ดูเพื่อนบ้านที่ใกล้ที่สุด 3 จุด แล้วใช้ผลโหวต
- สุดท้ายเราจะวัดความแม่นยำของโมเดลด้วย `score()`

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# สร้างโมเดลและฝึกกับข้อมูล
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

# วัดความแม่นยำ
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 🧠 แสดงการทำงานของ KNN ด้วยภาพและคำอธิบาย

### 🧪 การใช้ข้อมูลจริงทดลองใน 1D
**KNN แบบ 1 มิติ (1D)** คือการเลือกฟีเจอร์เพียงหนึ่งตัว เช่น `petal_length` มาคำนวณระยะห่าง

**กระบวนการทำงานของ KNN 1D**:
1. เลือกฟีเจอร์เดียวจากข้อมูล เช่น `petal_length`
2. วัดระยะห่าง (เช่น |x - xi| หรือ Euclidean)
3. เลือก k ตัวอย่างที่ใกล้ที่สุด (k = 3)
4. โหวตว่า label ของจุดใหม่ควรเป็นคลาสใด

แม้ว่าจะง่ายต่อการแสดงผลด้วยภาพ แต่ KNN 1D มักมีความแม่นยำน้อยกว่า เพราะข้อมูลอาจซ้อนกันระหว่างคลาส

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import ipywidgets as widgets
from IPython.display import display

# ฟีเจอร์ทั้งหมดในชุดข้อมูล
feature_options = list(X.columns)

# สร้าง widgets
feature_dropdown = widgets.Dropdown(
    options=feature_options,
    value='petal_length',
    description='Feature:',
)

k_slider = widgets.SelectionSlider(
    options=[k for k in range(1, 100, 2)],
    value=1,
    description='k:',
    continuous_update=False
)

def plot_knn_1d(feature_name, k):
    X_1d = X[[feature_name]].values
    x_new = X_test[[feature_name]].values
    y_true = y_test

    model_1d = KNeighborsClassifier(n_neighbors=k)
    model_1d.fit(X_1d, y)
    y_pred = model_1d.predict(x_new)

    correct = y_pred == y_true
    colors = ['green' if c else 'red' for c in correct]
    accuracy = correct.sum() / len(correct) * 100

    fig, ax = plt.subplots(figsize=(13, 3))

    # Training data by class
    ax.scatter(X_1d[y == 0], np.zeros_like(X_1d[y == 0]), c='purple', s=50, alpha=0.5, label='Iris-setosa')
    ax.scatter(X_1d[y == 1], np.zeros_like(X_1d[y == 1]), c='green',  s=50, alpha=0.5, label='Iris-versicolor')
    ax.scatter(X_1d[y == 2], np.zeros_like(X_1d[y == 2]), c='gold',   s=50, alpha=0.5, label='Iris-virginica')

    # X_test จุดใหม่
    ax.scatter(x_new[:, 0], np.zeros_like(x_new[:, 0]), c=colors, marker='x', s=80, label='X_test (✓ = green, ✗ = red)')

    ax.set_yticks([])
    ax.set_xlabel(feature_name.replace('_', ' ').title())
    ax.set_title(f'KNN (1D) with k = {k} on Feature: {feature_name}')
    ax.grid(True)

    # ✅ Legend ด้านล่างซ้าย
    ax.legend(loc='lower left', bbox_to_anchor=(0, -0.35), fontsize=10, frameon=True)

    # ✅ Accuracy ด้านล่างขวา
    plt.text(0.98, -0.42, f"Accuracy: {accuracy:.2f}%", transform=ax.transAxes, fontsize=12, ha='right')

    plt.tight_layout()
    plt.show()

# แสดง widgets และกราฟแบบ interactive
widgets.interact(plot_knn_1d, feature_name=feature_dropdown, k=k_slider);

### 📊 การใช้ข้อมูลจริงทดลองใน 2D
การวาดภาพนี้เลือกใช้ 2 จาก 4 ฟีเจอร์ เพื่อให้สามารถ plot เป็นกราฟ 2D ได้:

- ตัวอย่างในโค้ดนี้ใช้ `petal_length` (ความยาวกลีบดอกด้านใน) และ `petal_width` (ความกว้างกลีบดอกด้านใน)
- คุณสามารถเปลี่ยนไปใช้ฟีเจอร์อื่น ๆ เช่น `sepal_length`, `sepal_width` ได้ใน interactive version ด้านล่าง

> การเลือกฟีเจอร์มีผลต่อการแยกกลุ่มในกราฟ เพราะบางฟีเจอร์ช่วยแยกคลาสได้ชัดเจนกว่าฟีเจอร์อื่น

เราจะใช้ **matplotlib** ในการแสดงข้อมูล 2 มิติ (โดยเลือก 2 ฟีเจอร์) เพื่อให้เห็นว่า KNN ทำนายคลาสของจุดใหม่อย่างไร
- ใช้ฟีเจอร์: `petal length` กับ `petal width` ซึ่งแยก class ได้ดี
- วาดจุดใหม่ที่ต้องการทำนาย และแสดงเพื่อนบ้านที่ใกล้ที่สุด
- แสดงผลลัพธ์ว่า KNN เลือกคลาสใด

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

# ฟีเจอร์ทั้งหมด
feature_options = list(X.columns)

# Dropdown สำหรับเลือกฟีเจอร์ 2 ตัว
feature1_dropdown = widgets.Dropdown(options=feature_options, value='petal_length', description='Feature X:')
feature2_dropdown = widgets.Dropdown(options=feature_options, value='petal_width', description='Feature Y:')

# Slider สำหรับเลือกค่า k
k_slider = widgets.SelectionSlider(
    options=[k for k in range(1, 100, 2)],
    value=1,
    description='k:',
    continuous_update=False
)

# ฟังก์ชันวาดกราฟ
def plot_knn_2d(feature_x, feature_y, k):
    if feature_x == feature_y:
        print("กรุณาเลือกฟีเจอร์ที่ไม่ซ้ำกัน")
        return

    # เตรียมข้อมูล
    X_2d = X_train[[feature_x, feature_y]].values
    y_2d = y_train
    X_test_2d = X_test[[feature_x, feature_y]].values
    y_true = y_test

    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_2d, y_2d)
    y_pred = model.predict(X_test_2d)
    accuracy = (y_pred == y_true).sum() / len(y_true) * 100

    fig, ax = plt.subplots(figsize=(8, 6))

    # Training data
    ax.scatter(X_2d[y_2d == 0, 0], X_2d[y_2d == 0, 1], c='purple', s=50, alpha=0.6, label='Iris-setosa')
    ax.scatter(X_2d[y_2d == 1, 0], X_2d[y_2d == 1, 1], c='green',  s=50, alpha=0.6, label='Iris-versicolor')
    ax.scatter(X_2d[y_2d == 2, 0], X_2d[y_2d == 2, 1], c='gold',   s=50, alpha=0.6, label='Iris-virginica')

    # plot จุดจาก X_test และเชื่อมเพื่อนบ้าน
    for new_point in X_test_2d:
        nbrs = NearestNeighbors(n_neighbors=k)
        nbrs.fit(X_2d)
        distances, indices = nbrs.kneighbors([new_point])

        ax.scatter(new_point[0], new_point[1], c='black', marker='X', s=100)

        for idx in indices[0]:
            neighbor = X_2d[idx]
            ax.plot([new_point[0], neighbor[0]], [new_point[1], neighbor[1]], 'k--', alpha=0.3)

    ax.set_xlabel(feature_x.replace('_', ' ').title())
    ax.set_ylabel(feature_y.replace('_', ' ').title())
    ax.set_title(f'KNN (2D) Visualization with k = {k}')
    ax.grid(True)

    # ✅ Legend ด้านล่างซ้าย + accuracy อยู่ในกรอบด้วย
    legend_text = f"Accuracy: {accuracy:.2f}%"
    ax.legend(loc='lower right', fontsize=10, frameon=True, title=f"Accuracy: {accuracy:.2f}%", title_fontsize=10)

    plt.tight_layout()
    plt.show()

# แสดง widget ใหม่
widgets.interact(plot_knn_2d, feature_x=feature1_dropdown, feature_y=feature2_dropdown, k=k_slider);


### 🧊 การใช้ข้อมูลจริงทดลองใน 3D

- จุดใหม่ที่ต้องการทำนายจะแสดงเป็นเครื่องหมาย X สีดำ
- ใช้ `matplotlib` และ `Axes3D` ในการแสดงผล

In [ ]:
import plotly.graph_objs as go
import numpy as np
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import ipywidgets as widgets
from IPython.display import display

# ต้องมี X_train, X_test, y_train, y_test และ feature_options ก่อนหน้านี้
# ถ้ายังไม่มี ให้แน่ใจว่า import และแบ่ง dataset แล้วเรียบร้อย

# UI dropdowns
feature1_dropdown = widgets.Dropdown(options=feature_options, value='petal_length', description='Feature X:')
feature2_dropdown = widgets.Dropdown(options=feature_options, value='petal_width', description='Feature Y:')
feature3_dropdown = widgets.Dropdown(options=feature_options, value='sepal_length', description='Feature Z:')
k_slider = widgets.SelectionSlider(options=list(range(1, 16, 2)), value=3, description='k:')

# สีตาม class
class_color = {0: 'purple', 1: 'green', 2: 'gold'}

def plot_knn_3d_plotly(feature_x, feature_y, feature_z, k):
    if len({feature_x, feature_y, feature_z}) < 3:
        print("กรุณาเลือกฟีเจอร์ที่ไม่ซ้ำกันทั้ง 3 แกน")
        return

    X_train_3d = X_train[[feature_x, feature_y, feature_z]].values
    X_test_3d = X_test[[feature_x, feature_y, feature_z]].values
    y_train_ = y_train
    y_test_ = y_test

    # Train model
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train_3d, y_train_)
    y_pred = model.predict(X_test_3d)
    accuracy = (y_pred == y_test_).sum() / len(y_test_) * 100

    # สร้างกราฟ
    traces = []

    # 🔹 จุด Training set (ตาม class)
    for cls in np.unique(y_train_):
        cls_points = X_train_3d[y_train_ == cls]
        traces.append(go.Scatter3d(
            x=cls_points[:, 0], y=cls_points[:, 1], z=cls_points[:, 2],
            mode='markers',
            marker=dict(size=5, color=class_color[cls]),
            name=f'Class {cls} (train)'
        ))

    # 🔸 จุด Test set
    traces.append(go.Scatter3d(
        x=X_test_3d[:, 0], y=X_test_3d[:, 1], z=X_test_3d[:, 2],
        mode='markers',
        marker=dict(size=6, color='black', symbol='x'),
        name='X_test'
    ))

    # 🔻 เส้นปะแดงเชื่อมเพื่อนบ้าน
    nbrs = NearestNeighbors(n_neighbors=k)
    nbrs.fit(X_train_3d)

    for pt in X_test_3d:
        distances, indices = nbrs.kneighbors([pt])
        for idx in indices[0]:
            neighbor = X_train_3d[idx]
            traces.append(go.Scatter3d(
                x=[pt[0], neighbor[0]],
                y=[pt[1], neighbor[1]],
                z=[pt[2], neighbor[2]],
                mode='lines',
                line=dict(color='red', width=4, dash='dash'),
                showlegend=False
            ))

    # Layout
    layout = go.Layout(
        scene=dict(
            xaxis_title=feature_x,
            yaxis_title=feature_y,
            zaxis_title=feature_z
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        title=f"KNN 3D Visualization (k={k}) | Accuracy: {accuracy:.2f}%",
        width=1200,   # ✅ ความกว้างมากขึ้น
        height=500,   # ✅ ความสูงมากขึ้น
        showlegend=True
    )

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

# 🔧 เรียก UI interactive
widgets.interact(
    plot_knn_3d_plotly,
    feature_x=feature1_dropdown,
    feature_y=feature2_dropdown,
    feature_z=feature3_dropdown,
    k=k_slider
);

## 🧮 การใช้ข้อมูลจริงทดลองใน 4D
ในที่นี้เราจะอธิบายการทำงานของ KNN เมื่อใช้ฟีเจอร์ทั้ง 4 ตัว ผ่านการคำนวณแบบเมทริกซ์

**ข้อมูล training** (บางตัวอย่าง):
```
X_train = [
 [5.1, 3.5, 1.4, 0.2],  → Setosa
 [6.7, 3.0, 5.2, 2.3],  → Virginica
 [5.9, 3.2, 4.8, 1.8]   → Virginica
]
```
**ข้อมูลใหม่:**
```
x_new = [6.0, 2.9, 4.5, 1.5]
```

---

**สูตร Euclidean Distance:**
$$
\text{distance}(x, x_i) = \sqrt{\sum_{j=1}^n (x_j - x_{ij})^2}
$$

---

### 🔍 **ตารางการคำนวณระยะห่าง:**

| Training Index | วิธีคำนวณระยะห่างแบบละเอียด          | Label     | Distance to `x_new`  |
|----------------|----------------------------------|-----------|----------------------|
| 0              | $\sqrt{(6.0-5.1)^2+(2.9-3.5)^2+(4.5-1.4)^2+(1.5-0.2)^2}$            | Setosa    | 3.99                 |
| 1              | $\sqrt{(6.0-6.7)^2+(2.9-3.0)^2+(4.5-5.2)^2+(1.5-2.3)^2}$            | Virginica | 1.28                 |
| 2              | $\sqrt{(6.0-5.9)^2+(2.9-3.2)^2+(4.5-4.8)^2+(1.5-1.8)^2}$            | Virginica | 0.45                 |

### 📌 **ผลการทำนาย (k = 3):**
- เพื่อนบ้านที่ใกล้ที่สุด: Index 2, 1, 0
- Labels = `[2, 2, 0]`
- Mode = `2` → 🔮 **ทำนายว่าเป็น Virginica**

---

### ✅ สรุป:
> ด้วยการคำนวณระยะห่างในข้อมูล 4 มิติ จุดใหม่ `[6.0, 2.9, 4.5, 1.5]`  
> มีเพื่อนบ้านใกล้สุด 3 ตัว ซึ่ง 2 ใน 3 เป็น `Virginica`  
> โมเดล KNN จึงทำนายว่าดอกไม้ดอกนี้คือ: ✅ **Virginica**

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

# UI dropdowns
feature_dropdowns = [widgets.Dropdown(options=feature_options, description=label)
                     for label in ['X Axis:', 'Y Axis:', 'Size:', 'Color:']]

k_slider = widgets.SelectionSlider(
    options=list(range(1, 100, 2)), value=3, description='k:', continuous_update=False
)

def plot_knn_4d(f1, f2, f3, f4, k):
    if len({f1, f2, f3, f4}) < 4:
        print("กรุณาเลือกฟีเจอร์ไม่ซ้ำกันทั้ง 4 ตัว")
        return

    # 🔹 Train KNN
    X_train_knn = X_train[[f1, f2, f3, f4]].values
    X_test_knn = X_test[[f1, f2, f3, f4]].values
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train_knn, y_train)
    y_pred = model.predict(X_test_knn)
    acc = (y_pred == y_test).mean() * 100

    # 🔸 Prepare DataFrame
    df_plot = X_test.copy()
    df_plot['size'] = df_plot[f3]
    df_plot['color'] = df_plot[f4]
    df_plot['true_label'] = y_test
    df_plot['pred_label'] = y_pred
    df_plot['correct'] = df_plot['true_label'] == df_plot['pred_label']

    class_names = {0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'}
    class_colors = ['purple', 'green', 'gold']

    plt.figure(figsize=(10, 7))

    # 🔹 Plot ทายถูก (marker = o)
    for cls in sorted(df_plot['true_label'].unique()):
        df_cls = df_plot[(df_plot['true_label'] == cls) & (df_plot['correct'] == True)]
        plt.scatter(
            df_cls[f1], df_cls[f2],
            s=df_cls['size'] * 30,
            c=class_colors[cls],
            marker='o', alpha=0.8, edgecolors='black',
            label=f"✔ Class {cls} ({class_names.get(cls)})"
        )

    # 🔻 Plot ทายผิด (marker = x)
    for cls in sorted(df_plot['true_label'].unique()):
        df_cls = df_plot[(df_plot['true_label'] == cls) & (df_plot['correct'] == False)]
        if not df_cls.empty:
            plt.scatter(
                df_cls[f1], df_cls[f2],
                s=df_cls['size'] * 30,
                c=class_colors[cls],
                marker='x', alpha=0.9, linewidths=2,
                label=f"✘ Class {cls} ({class_names.get(cls)})"
            )

    # 🔧 Styling
    plt.xlabel(f1.replace('_', ' ').title())
    plt.ylabel(f2.replace('_', ' ').title())
    plt.title(f"KNN 4D Bubble Matrix (k={k}) | Accuracy: {acc:.2f}%\n"
              f"X = {f1}, Y = {f2}, Size = {f3}, Color = Class, Shape = Prediction Correctness")
    plt.grid(True)
    plt.legend(loc='upper right', title="Legend")
    plt.show()

# interactive widget
widgets.interact(
    plot_knn_4d,
    f1=feature_dropdowns[0],
    f2=feature_dropdowns[1],
    f3=feature_dropdowns[2],
    f4=feature_dropdowns[3],
    k=k_slider
);